In [ ]:
from tqdm import tqdm
import s3fs
import os

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "cthiounn2"
fs.ls(BUCKET)

files=['pokemon.parquet','pokemon_name.csv']
for file in tqdm(files):
    with fs.open(f'{BUCKET}/{file}', mode="rb") as file_in, open(file,"wb") as file_out:
            file_out.write(file_in.read())

In [ ]:
import pandas as pd
df= pd.read_parquet("pokemon.parquet")

In [ ]:
df_pokemon_non_mega=df[~df.file_name.str.contains("-")]

In [ ]:
df_pokemon_non_mega[["numero","filetype"]]=df_pokemon_non_mega["file_name"].str.split(pat=".",expand=True)

In [ ]:
df_pokemon_non_mega['numeroint']=df_pokemon_non_mega['numero'].apply(int)

In [ ]:
df_pokemon_non_mega_numenc=df_pokemon_non_mega[['numeroint','encoding']]
df_pokemon_non_mega_numenc=df_pokemon_non_mega_numenc.reset_index(drop=True)
df_pokemon_non_mega_numenc=df_pokemon_non_mega_numenc.set_index('numeroint')

In [ ]:
# English name https://github.com/veekun/pokedex/blob/master/pokedex/data/csv/pokemon.csv
# French name
df_name= pd.read_csv("pokemon_name.csv")


In [ ]:
df_name_keep=df_name[['Numéro', 'Nom français']]
df_name_keep=df_name_keep.reset_index(drop=True)
df_name_keep=df_name_keep.set_index('Numéro')

In [ ]:
df_fusion=df_pokemon_non_mega_numenc.merge(df_name_keep,how="left",right_index=True,left_index=True)

In [ ]:
df_fusion_ordre=df_fusion[['Nom français','encoding']]

In [ ]:
df_fusion_ordre